In [ ]:
# ==========================================
# Country: Ethiopia
# Task: Task 2 - Data Profiling, Cleaning & EDA
# Branch: eda-ethiopia
# ==========================================

# 1️⃣ Imports
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Optional: For wind rose plots
# !pip install windrose
from windrose import WindroseAxes

# ------------------------------------------
# 2️⃣ Load Dataset
df = pd.read_csv("data/ethiopia_solar.csv")
df.head()

# ------------------------------------------
# 3️⃣ Summary Statistics & Missing Values
print("=== Numeric Summary ===")
display(df.describe())

missing_values = df.isna().sum()
missing_percent = (df.isna().mean() * 100).round(2)
cols_nulls_gt5 = missing_percent[missing_percent > 5]

print("Missing values per column:\n", missing_values)
print("\nPercentage of missing values:\n", missing_percent)
print("\nColumns with >5% missing values:\n", cols_nulls_gt5)

# ------------------------------------------
# 4️⃣ Outlier Detection & Basic Cleaning
key_columns = ["GHI", "DNI", "DHI", "ModA", "ModB", "WS", "WSgust"]

# Fill missing values with median
for col in key_columns:
    df[col].fillna(df[col].median(), inplace=True)

# Compute Z-scores
z_scores = np.abs(stats.zscore(df[key_columns]))
z_df = pd.DataFrame(z_scores, columns=key_columns)

# Flag rows with |Z|>3
outliers = (z_df > 3).any(axis=1)
print(f"Number of outlier rows: {outliers.sum()}")

# Create a cleaning flag
df["clean_flag"] = ~outliers

# Drop outliers
df_clean = df[df["clean_flag"]].copy()

# ------------------------------------------
# 5️⃣ Time Series Analysis
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Line plots for GHI, DNI, DHI, Tamb
plt.figure(figsize=(12,6))
plt.plot(df_clean["Timestamp"], df_clean["GHI"], label="GHI")
plt.plot(df_clean["Timestamp"], df_clean["DNI"], label="DNI")
plt.plot(df_clean["Timestamp"], df_clean["DHI"], label="DHI")
plt.plot(df_clean["Timestamp"], df_clean["Tamb"], label="Tamb")
plt.xlabel("Timestamp")
plt.ylabel("Value")
plt.title("Time Series of Solar Variables")
plt.legend()
plt.show()

# Optional: Monthly averages
df_clean["month"] = df_clean["Timestamp"].dt.month
monthly_avg = df_clean.groupby("month")[["GHI", "DNI", "DHI", "Tamb"]].mean()
monthly_avg.plot(kind="bar", figsize=(12,6))
plt.title("Monthly Average Solar & Temp Variables")
plt.show()

# ------------------------------------------
# 6️⃣ Cleaning Impact
df_clean.groupby("clean_flag")[["ModA", "ModB"]].mean().plot(kind="bar", figsize=(8,5))
plt.title("Impact of Cleaning on Module Sensors")
plt.show()

# ------------------------------------------
# 7️⃣ Correlation & Relationship Analysis
plt.figure(figsize=(10,6))
sns.heatmap(df_clean[["GHI", "DNI", "DHI", "ModA", "ModB"]].corr(), annot=True, fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

# Scatter plots
sns.scatterplot(data=df_clean, x="WS", y="GHI")
sns.scatterplot(data=df_clean, x="WSgust", y="GHI")
sns.scatterplot(data=df_clean, x="WD", y="GHI")
sns.scatterplot(data=df_clean, x="RH", y="Tamb")
sns.scatterplot(data=df_clean, x="RH", y="GHI")
plt.show()

# ------------------------------------------
# 8️⃣ Wind & Distribution Analysis
# Wind rose
plt.figure(figsize=(8,8))
ax = WindroseAxes.from_ax()
ax.bar(df_clean["WD"], df_clean["WS"], normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.title("Wind Rose (WS/WD)")
plt.show()

# Histogram for GHI
df_clean["GHI"].hist(bins=50, figsize=(10,5))
plt.title("GHI Distribution")
plt.show()

# Histogram for WS
df_clean["WS"].hist(bins=50, figsize=(10,5))
plt.title("WS Distribution")
plt.show()

# ------------------------------------------
# 9️⃣ Temperature & Humidity Analysis
sns.scatterplot(data=df_clean, x="RH", y="Tamb")
plt.title("Relative Humidity vs Temperature")
plt.show()

sns.scatterplot(data=df_clean, x="RH", y="GHI")
plt.title("Relative Humidity vs GHI")
plt.show()

# ------------------------------------------
# 🔟 Bubble Chart
plt.figure(figsize=(10,6))
plt.scatter(df_clean["Tamb"], df_clean["GHI"], s=df_clean["RH"], alpha=0.5)
plt.xlabel("Tamb")
plt.ylabel("GHI")
plt.title("GHI vs Tamb (bubble size=RH)")
plt.show()

# ------------------------------------------
# 1️⃣1️⃣ Export Cleaned Data
os.makedirs("data", exist_ok=True)
df_clean.to_csv("data/ethiopia_clean.csv", index=False)
print("Cleaned dataset exported to data/ethiopia_clean.csv (do NOT commit this file)")

